In [76]:
from dotenv import load_dotenv
from core.crawler import TweetCrawler
from psycopg.rows import dict_row
from psycopg_pool import ConnectionPool
from twscrape import Tweet, User
import pandas as pd
import psycopg
import simplejson
import os
import re
load_dotenv('.env')

True

In [6]:
from core.config import POSTGRE_DB_CONN

In [10]:
connection_pool = ConnectionPool(conninfo=POSTGRE_DB_CONN, kwargs={"row_factory": dict_row})

In [11]:
data = pd.read_json('./crawling_history/2023-05-09_00:00:00_UTC.json')

In [25]:
tweets = []
for x in data.iloc():
  tweets.append(Tweet(**x.to_dict()))

In [77]:
users_tweets = []
cleaned_tweets = []

for tweet in tweets:
    tweet:Tweet
    regex_ads = r"(ads)|(advertiser)"
    is_ads = [
        *re.findall(regex_ads, tweet.sourceUrl.lower()),
        *re.findall(regex_ads, tweet.sourceLabel.lower()),
        *re.findall(regex_ads, tweet.source.lower()),
    ]
    if not is_ads:
        pass
    else:
        continue
    twtype = "tweet"

    if tweet.quotedTweet:
        twtype = "retweet"
    if tweet.inReplyToUser and tweet.inReplyToTweetId:
        twtype = "replied_to"
    
    cleaned_tweets.append(
        [tweet.id_str, tweet.rawContent, tweet.user.id_str, tweet.sourceLabel, simplejson.dumps(tweet.json(), ignore_nan=True), tweet.date, twtype]
    )

    users_tweets.append(
        [tweet.user.id_str, tweet.user.username, tweet.user.displayname, simplejson.dumps(tweet.user.json(), ignore_nan=True), tweet.user.created]
    )

In [51]:
crawler = TweetCrawler(connection_pool)

In [78]:
def divide_chunks(l, n): 
    for i in range(0, len(l), n):  
        yield l[i:i + n] 

In [80]:
for x in list(divide_chunks(cleaned_tweets, 100)):
  crawler.insert_bulk_tweet(x)